In [ ]:
import pyautogui
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from dotenv import load_dotenv
import os
import re


# Carrega as variáveis do .env
dotenv_path = "../data/secure/.env"
load_dotenv(dotenv_path)

EMAIL = os.getenv("EMAIL")
SENHA = os.getenv("SENHA")

# Caminho para o chromedriver
chromedriver_path = r"C:\Users\User\Desktop\Repositorios\Automações\src\others\chromedriver.exe"

# Configurações do Chrome
options = Options()
options.add_argument("--start-maximized")  # opcional: inicia em tela cheia

# Inicia o serviço e o navegador
service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service, options=options)

# Abre a página de login
driver.get("https://app.simples.vet/login/login.php")

# Espera a página carregar
time.sleep(2)

# Preenche os campos de login
driver.find_element(By.ID, "l_usu_var_email").send_keys(EMAIL)
driver.find_element(By.ID, "l_usu_var_senha").send_keys(SENHA)

# Pressiona Enter para enviar o formulário
driver.find_element(By.ID, "l_usu_var_senha").send_keys(Keys.RETURN)

# Aguarda o login ser processado e o botão "Vendas" estar disponível
vendas_element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//a[.//span[text()='Vendas ']]"))
)

# Clica no botão "Vendas"
vendas_element.click()

# Espera e clica no link "Consulta vendas"
consulta_vendas_link = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//a[@class='link-menu' and contains(text(), 'Consulta vendas')]"))
)
consulta_vendas_link.click()

time.sleep(2)

driver.find_element(By.ID, "p__ven_var_chave").send_keys(EMAIL)

df = pd.read_csv("../data/input/filtered/filtered_df.csv")

for index, row in df.iterrows():
    id_value = str(row['ID'])  # pega o id

    # Espera o input aparecer e limpa antes de digitar
    input_id = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "p__ven_var_chave"))
    )
    input_id.clear()
    input_id.send_keys(id_value)

    # Aqui você pode colocar o código pra clicar no botão pesquisar
    # Exemplo:
    # btn_pesquisar = driver.find_element(By.ID, "id_do_botao")
    # btn_pesquisar.click()

    # Dá um tempinho para o site processar
    driver.find_element(By.ID, "p__ven_var_chave").send_keys(Keys.RETURN)
    time.sleep(2)

    try:
        # Espera o elemento aparecer depois da pesquisa
        ficha_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ficha"))
        )
        ficha_text = ficha_element.text  # Exemplo: "(16603)"
        # Extrai só o número dentro dos parênteses
        cliente_num = re.search(r"\((\d+)\)", ficha_text).group(1)
    except Exception as e:
        print(f"Erro ao capturar ficha para ID {id_value}: {e}")
        cliente_num = None

    # Atualiza o dataframe na linha atual
    df.loc[index, 'cliente'] = cliente_num


    # Depois que terminar, segue para o próximo id

df['VALOR'] = df['VALOR'].str.replace(',', '.').astype(float)

# 2. Agrupar por cliente e somar VALOR
df_resumo = df.groupby('cliente', as_index=False)['VALOR'].sum()

# Se quiser, pode ordenar pelo cliente ou valor
df_resumo = df_resumo.sort_values(by='cliente')

clientes_btn = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//a[@class='link-menu']//span[text()='Clientes']"))
)
clientes_btn.click()

display(df_resumo)


In [ ]:
import pyautogui
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from dotenv import load_dotenv
import os
import re

# Carrega as variáveis do .env
dotenv_path = "../data/secure/.env"
load_dotenv(dotenv_path)

EMAIL = os.getenv("EMAIL")
SENHA = os.getenv("SENHA")

chromedriver_path = r"C:\Users\User\Desktop\Repositorios\Automações\src\others\chromedriver.exe"

options = Options()
options.add_argument("--start-maximized")
service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service, options=options)

driver.get("https://app.simples.vet/login/login.php")
time.sleep(2)

driver.find_element(By.ID, "l_usu_var_email").send_keys(EMAIL)
driver.find_element(By.ID, "l_usu_var_senha").send_keys(SENHA)
driver.find_element(By.ID, "l_usu_var_senha").send_keys(Keys.RETURN)

# Acessa o menu de vendas > consulta vendas
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//a[.//span[text()='Vendas ']]"))
).click()

WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//a[@class='link-menu' and contains(text(), 'Consulta vendas')]"))
).click()

time.sleep(2)

driver.find_element(By.ID, "p__ven_var_chave").send_keys(EMAIL)

df = pd.read_csv("../data/input/filtered/filtered_df.csv")

# Coleta o número do cliente de cada ID
for index, row in df.iterrows():
    id_value = str(row['ID'])

    input_id = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "p__ven_var_chave"))
    )
    input_id.clear()
    input_id.send_keys(id_value)
    input_id.send_keys(Keys.RETURN)
    time.sleep(2)

    try:
        ficha_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ficha"))
        )
        ficha_text = ficha_element.text
        cliente_num = re.search(r"\((\d+)\)", ficha_text).group(1)
    except Exception as e:
        print(f"Erro ao capturar ficha para ID {id_value}: {e}")
        cliente_num = None

    df.loc[index, 'cliente'] = cliente_num

# Agrupa por cliente
df['VALOR'] = df['VALOR'].str.replace(',', '.').astype(float)
df_resumo = df.groupby('cliente', as_index=False)['VALOR'].sum()
df_resumo = df_resumo.sort_values(by='cliente')
df_resumo['CPF'] = ""  # Cria coluna para preencher depois

clientes_btn = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//a[@class='link-menu' and .//span[text()='Clientes']]"))
)
clientes_btn.click()

# Itera sobre os clientes únicos e busca o CPF
for index, row in df_resumo.iterrows():
    cliente_id = str(row['cliente'])

    # Espera input aparecer e preenche
    input_nome = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "p__pes_var_nome"))
    )
    input_nome.clear()
    input_nome.send_keys(cliente_id)
    input_nome.send_keys(Keys.RETURN)
    time.sleep(2)

    try:
        # Clica no primeiro resultado da lista
        elemento = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//tbody[@id='bodyLoad']//a[contains(@class, 'linkAnimalLista')]"))
        )
        elemento.click()
        time.sleep(2)

        # Tenta encontrar o CPF
        try:
            cpf_element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, "//li[contains(text(), 'CPF:')]"))
            )
            cpf_text = cpf_element.text
            cpf_value = re.search(r"CPF:\s+([\d.-]+)", cpf_text).group(1)
        except:
            cpf_value = "000.000.000-00"

        df_resumo.loc[index, 'CPF'] = cpf_value

        # Fecha a tela do cliente
        fechar_btn = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.ID, "v__btn_fechar_topo"))
        )
        fechar_btn.click()
        time.sleep(1)

        # Clica de novo no menu "Clientes" para voltar
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@class='link-menu']//span[text()='Clientes']"))
        ).click()

    except Exception as e:
        print(f"Erro ao processar cliente {cliente_id}: {e}")
        df_resumo.loc[index, 'CPF'] = "000.000.000-00"

# Exibe o DataFrame final
print(df_resumo)


In [ ]:
import re
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Exemplo de DataFrame simplificado (substitua pelo seu df_resumo real)
df_resumo = pd.DataFrame({
    'cliente': ['10529', '16603', '13283'],  # ids de clientes para testar
    'VALOR': [300.00, 150.00, 400.00],
    'CPF': ['']*3
})

# Já deve estar no menu Clientes (ou abra manualmente)
# driver.get("URL_do_menu_clientes")  # Se quiser forçar a navegação

for index, row in df_resumo.iterrows():
    cliente_id = str(row['cliente'])
    print(f"Processando cliente {cliente_id} (linha {index})...")

    try:
        # Espera input aparecer e preenche
        input_nome = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.ID, "p__pes_var_nome"))
        )
        input_nome.clear()
        input_nome.send_keys(cliente_id)
        input_nome.send_keys(Keys.RETURN)
        print("Pesquisa enviada.")
        time.sleep(3)

        # Clica no primeiro resultado da lista
        elemento = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//tbody[@id='bodyLoad']//a[contains(@class, 'linkAnimalLista')]"))
        )
        elemento.click()
        print("Cliente clicado.")
        time.sleep(3)

        # Clica no span para abrir edição do responsável (CPF)
        div_responsavel = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.ID, "divDadosProprietario"))
        )
        div_responsavel.click()
        print("Seção de responsável aberta.")
        time.sleep(2)

        # Busca o input do CPF e pega o valor
        try:
            cpf_input = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "pes_var_cpf"))
            )
            cpf_value = cpf_input.get_attribute("value").strip()
            if not cpf_value:
                cpf_value = "000.000.000-00"
        except Exception as e:
            print(f"Erro ao buscar CPF do cliente {cliente_id}: {e}")
            cpf_value = "000.000.000-00"

        df_resumo.loc[index, 'CPF'] = cpf_value
        print(f"CPF obtido: {cpf_value}")

        # Fecha a tela do cliente
        fechar_btn = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.ID, "v__btn_fechar_topo"))
        )
        fechar_btn.click()
        print("Tela fechada.")
        time.sleep(2)

        # Tenta clicar no menu "Clientes" com retry para garantir clique
        for attempt in range(3):
            try:
                clientes_btn = WebDriverWait(driver, 15).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "a.link-menu[data-id='118']"))
                )
                clientes_btn.click()
                print("Voltando para menu Clientes.")
                time.sleep(2)
                break  # saiu do loop se clicou com sucesso
            except Exception as e:
                print(f"Tentativa {attempt + 1} para clicar no menu Clientes falhou: {e}")
                time.sleep(2)
        else:
            print("Não conseguiu clicar no menu Clientes após 3 tentativas.")

    except Exception as e:
        print(f"Erro ao processar cliente {cliente_id}: {repr(e)}")
        df_resumo.loc[index, 'CPF'] = "000.000.000-00"

print("\nResultado final:")
print(df_resumo)


In [ ]:
import re
import time
import os
import pandas as pd
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- Configurações iniciais ---

# Carrega variáveis do arquivo .env
dotenv_path = "../data/secure/.env"
load_dotenv(dotenv_path)
EMAIL = os.getenv("EMAIL")
SENHA = os.getenv("SENHA")

# Caminho do chromedriver
chromedriver_path = r"C:\Users\User\Desktop\Repositorios\Automações\src\others\chromedriver.exe"

# Configurações do navegador Chrome
options = Options()
options.add_argument("--start-maximized")
service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service, options=options)

# --- Login e navegação até Consulta Vendas ---

driver.get("https://app.simples.vet/login/login.php")
time.sleep(2)

driver.find_element(By.ID, "l_usu_var_email").send_keys(EMAIL)
driver.find_element(By.ID, "l_usu_var_senha").send_keys(SENHA)
driver.find_element(By.ID, "l_usu_var_senha").send_keys(Keys.RETURN)

# Espera o menu "Vendas" carregar e clica
WebDriverWait(driver, 15).until(
    EC.element_to_be_clickable((By.XPATH, "//a[.//span[text()='Vendas ']]"))
).click()

# Clica em "Consulta vendas"
WebDriverWait(driver, 15).until(
    EC.element_to_be_clickable((By.XPATH, "//a[@class='link-menu' and contains(text(), 'Consulta vendas')]"))
).click()

time.sleep(2)

# --- Carrega DataFrame original e faz busca por ID na consulta vendas ---

df = pd.read_csv("../data/input/filtered/filtered_df.csv")

for index, row in df.iterrows():
    id_value = str(row['ID'])

    input_id = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.ID, "p__ven_var_chave"))
    )
    input_id.clear()
    input_id.send_keys(id_value)
    input_id.send_keys(Keys.RETURN)
    time.sleep(2)

    try:
        ficha_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ficha"))
        )
        ficha_text = ficha_element.text  # exemplo: "(16603)"
        cliente_num = re.search(r"\((\d+)\)", ficha_text).group(1)
    except Exception as e:
        print(f"Erro ao capturar ficha para ID {id_value}: {e}")
        cliente_num = None

    df.loc[index, 'cliente'] = cliente_num

# Agrupa e soma valores por cliente
df['VALOR'] = df['VALOR'].str.replace(',', '.').astype(float)
df_resumo = df.groupby('cliente', as_index=False)['VALOR'].sum()
df_resumo = df_resumo.sort_values(by='cliente')
df_resumo['CPF'] = ""  # coluna para CPF

# --- Navega para o menu Clientes ---

# Clica no menu "Clientes" usando o seletor do botão confiável
clientes_btn = WebDriverWait(driver, 15).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "a.link-menu[data-id='118']"))
)
clientes_btn.click()
time.sleep(2)

# --- Loop para buscar CPF para cada cliente ---

for index, row in df_resumo.iterrows():
    cliente_id = str(row['cliente'])
    print(f"Processando cliente {cliente_id} (linha {index})...")

    try:
        # Preenche campo de busca por cliente
        input_nome = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.ID, "p__pes_var_nome"))
        )
        input_nome.clear()
        input_nome.send_keys(cliente_id)
        input_nome.send_keys(Keys.RETURN)
        print("Pesquisa enviada.")
        time.sleep(3)

        # Clica no primeiro resultado da lista
        elemento = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//tbody[@id='bodyLoad']//a[contains(@class, 'linkAnimalLista')]"))
        )
        elemento.click()
        print("Cliente clicado.")
        time.sleep(3)

        # Clica no span para abrir edição do responsável (CPF)
        div_responsavel = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.ID, "divDadosProprietario"))
        )
        div_responsavel.click()
        print("Seção de responsável aberta.")
        time.sleep(2)

        # Busca o input do CPF e pega o valor
        try:
            cpf_input = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "pes_var_cpf"))
            )
            cpf_value = cpf_input.get_attribute("value").strip()
            if not cpf_value:
                cpf_value = "000.000.000-00"
        except Exception as e:
            print(f"Erro ao buscar CPF do cliente {cliente_id}: {e}")
            cpf_value = "000.000.000-00"

        df_resumo.loc[index, 'CPF'] = cpf_value
        print(f"CPF obtido: {cpf_value}")

        # Fecha a tela do cliente
        fechar_btn = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.ID, "v__btn_fechar_topo"))
        )
        fechar_btn.click()
        print("Tela fechada.")
        time.sleep(2)

        # Tenta clicar no menu "Clientes" com retry para garantir clique
        for attempt in range(3):
            try:
                clientes_btn = WebDriverWait(driver, 15).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "a.link-menu[data-id='118']"))
                )
                clientes_btn.click()
                print("Voltando para menu Clientes.")
                time.sleep(2)
                break  # saiu do loop se clicou com sucesso
            except Exception as e:
                print(f"Tentativa {attempt + 1} para clicar no menu Clientes falhou: {e}")
                time.sleep(2)
        else:
            print("Não conseguiu clicar no menu Clientes após 3 tentativas.")

    except Exception as e:
        print(f"Erro ao processar cliente {cliente_id}: {repr(e)}")
        df_resumo.loc[index, 'CPF'] = "000.000.000-00"

print("\nResultado final:")
print(df_resumo)

# Opcional: salvar resultado em CSV
df_resumo.to_csv("../data/output/clientes_com_cpf.csv", index=False)
